# Synthetic dataset generator

In [1]:

import re
import os
import sys
import io
import json
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
from pathlib import Path
from datetime import datetime
import requests
import subprocess
from IPython.display import Markdown, display, update_display

In [2]:
# Initialize

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"

openai = OpenAI()
claude = anthropic.Anthropic()

In [3]:
system_message = """
                    You are a dataset generation assistant specializing in creating high-quality synthetic datasets for AI training.
                    Ensure data is realistic, diverse, and domain-specific.
                    Do not include personally identifiable information (PII).
                    Maintain logical consistency in relationships between fields.
                """

In [10]:
def generate_user_prompt_tabular(business_problem, dataset_format, file_format, num_samples):

    user_message = f"""
    The business problem is: {business_problem}. \n
    The dataset is expected to be in {dataset_format}. 
    For the dataset types such as tabular or time series implement python code for creating the dataset.
    If the generated dataset contains several entities, i.e. products, users, write the output for these entities into separate files. 
    The dependencies for python code should include only standard python libraries such as numpy, pandas and built-in libraries. 
    The output dataset is stored as a {file_format} file and contains {num_samples} samples. \n    
    """

def generate_user_prompt_text(business_problem, dataset_format, file_format):
    
    user_message = f"""
    The business problem is: {business_problem}. \n
    The dataset is expected to be in {dataset_format}. 
    For the text type return the generated dataset and the python code to write the output to the files.
    If the generated dataset contains several entities, i.e. products, users, write the output for these entities into separate files. 
    The dependencies for python code should include only standard python libraries such as numpy, pandas and built-in libraries. 
    The output dataset is stored as a {file_format} file. \n    
    """

    return user_message

def select_user_prompt(business_problem, dataset_format, file_format, num_samples):
    user_prompt = generate_user_prompt_text(business_problem, dataset_format, file_format)
    if dataset_format in ["Tabular", "Time-series"]:
        user_prompt = generate_user_prompt_tabular(business_problem, dataset_format, file_format, num_samples)
    return user_prompt
    

In [11]:
def stream_gpt(business_problem, dataset_format, file_format, num_samples):
    user_prompt = select_user_prompt(business_problem, dataset_format, file_format, num_samples)

    stream = openai.chat.completions.create(
                model=OPENAI_MODEL,
                messages= [
                    {"role": "system", "content":system_message},
                    {"role": "user", "content": user_prompt }
                    ],
                stream=True
            )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response

    return response

def stream_claude(business_problem, dataset_format, file_format, num_samples):
    user_prompt = select_user_prompt(
                    business_problem, dataset_format, file_format, num_samples
                )
    result = claude.messages.stream(
            model=CLAUDE_MODEL,
            max_tokens=2000,
            system=system_message,
            messages=[
                {"role" : "user", "content": user_prompt }
            ]
        )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply
        return reply
        

In [12]:
def generate_dataset(business_problem, dataset_format, file_format, num_samples, model):
    if model == "GPT":
        result = stream_gpt(business_problem, dataset_format, file_format, num_samples)
    elif model == "Claude":
        result = stream_claude(business_problem, dataset_format, file_format, num_samples)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far
    return result

In [13]:

def extract_code(text):
    # Regular expression to find text between ``python and ``
    match = re.search(r"```python(.*?)```", text, re.DOTALL)

    if match:
        code = match.group(0).strip()  # Extract and strip extra spaces
    else:
        code = ""
        print("No matching substring found.")

    return code.replace("```python\n", "").replace("```", "")


def execute_code_in_virtualenv(text, python_interpreter=sys.executable):
    """
    Execute the given Python code string within the specified virtual environment.
    
    Args:
    - code_str: str, the Python code to execute.
    - venv_dir: str, the directory path to the virtual environment created by pipenv.
    """
    # Construct the full path to the Python interpreter in the virtual environment
    # python_interpreter = f"{venv_dir}/bin/python"

    # Check if executing within the specified virtual environment interpreter
    if not python_interpreter:
        raise EnvironmentError("Python interpreter not found in the specified virtual environment.")

    # Prepare the command to execute the code
    code_str = extract_code(text)
    command = [python_interpreter, '-c', code_str]

    # Execute the command
    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Output:", result.stdout)
        print("Errors:", result.stderr)
    except subprocess.CalledProcessError as e:
        print(f"An error occurred while executing the code: {e}")
    return result.stdout

No matching substring found.
Output: 
Errors: 


''

# Gradio UI

In [16]:
with gr.Blocks() as ui:
    gr.Markdown("## Create a dataset for a business problem")
    with gr.Column():
        business_problem = gr.Textbox(label="Business problem", lines=2)
        dataset_type = gr.Dropdown(
            ["Tabular", "Time-series", "Text"], label="Dataset modality"
        )
        dataset_format = gr.Dropdown(["JSON", "csv", "parquet", "Markdown"], label="Output format")
        num_samples = gr.Number(label="Number of samples (for tabular and time-series data)", value=10, precision=0)
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
    with gr.Row():
        dataset_run = gr.Button("Create a dataset")
        code_run = gr.Button("Execute code for a dataset")
    with gr.Row():
        dataset_out = gr.Textbox(label="Generated Dataset")
        code_out = gr.Textbox(label="Executed code")
    dataset_run.click(
        generate_dataset,
        inputs=[business_problem, dataset_type, dataset_format, num_samples, model],
        outputs=[dataset_out]
    )
    code_run.click(execute_code_in_virtualenv, inputs=[dataset_out], outputs=[code_out])

In [17]:
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


Output: Generated synthetic customer profiles and saved to synthetic_customer_profiles.json.

Errors: 


# Sample usage
<img src="images/data_set_generator.png" width="700" height="500" style="display: block;" />

# Sample data generated
[customer_data.json]()